### Tasks/Activities List
Your code should contain the following activities and analyses:
- Collect the data from the zip file linked here.
- Split the dataset into dependent and independent variables.
- Perform train, test split.
- Apply StandardScaler() to the train and test dependent variable.
- Fit the best parameters.
- Model Prediction
- Model Validation Statistics
- Create a FastAPI app
- Create routes for getting predictions
- Create a route for retraining the model


In [1]:
import pandas as pd 
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,f1_score,recall_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model  import  LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_predict
import os
# pipelines
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
import warnings
warnings.filterwarnings("ignore")
print("All libraries sucessfully loaded ")

All libraries sucessfully loaded 


In [2]:
df=pd.read_csv("tel_churn.csv")
#data preview 
pd.set_option('display.max_columns', None)
df.head()

,Unnamed: 0,tenure,age,address,income,employ,reside,lninc,churn,total_monthly_bill,total_bill,total_log_bill,region_Zone 1,region_Zone 2,region_Zone 3,marital_Married,marital_Unmarried,ed_College degree,ed_Did not complete high school,ed_High school degree,ed_Post-undergraduate degree,ed_Some college,retire_No,retire_Yes,gender_Female,gender_Male,tollfree_No,tollfree_Yes,equip_No,equip_Yes,callcard_No,callcard_Yes,wireless_No,wireless_Yes,multline_No,multline_Yes,voice_No,voice_Yes,pager_No,pager_Yes,internet_No,internet_Yes,callid_No,callid_Yes,callwait_No,callwait_Yes,forward_No,forward_Yes,confer_No,confer_Yes,ebill_No,ebill_Yes,custcat_Basic service,custcat_E-service,custcat_Plus service,custcat_Total service
0,0,13,44,9,64,5,2,4.158883,1,11.20,147.45,3.323236,0,1,0,1,0,1,0,0,0,0,1,0,0,1,1,0,1,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0,1,0,0,0
1,1,11,33,7,136,5,6,4.912655,1,76.10,758.80,10.813882,0,0,1,1,0,0,0,0,1,0,1,0,0,1,0,1,1,0,0,1,0,1,1,0,0,1,0,1,1,0,0,1,0,1,0,1,0,1,1,0,0,0,0,1
2,2,68,52,24,116,29,2,4.753590,0,66.40,4697.80,9.198539,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,1,0,0,1,1,0,1,0,1,0,1,0,1,0,0,1,0,1,1,0,0,1,1,0,0,0,1,0
3,3,33,33,12,33,0,1,3.496508,1,9.45,288.80,2.246015,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0
4,4,23,30,9,30,2,4,3.401197,0,6.30,157.05,1.840550,0,1,0,1,0,0,1,0,0,0,1,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,0,1,0,1,1,0,0,0,1,0


In [3]:
df=df.drop('Unnamed: 0',axis=1)

In [4]:
# Splitting the independent and dependent variables

x = df.drop('churn', axis=1)# feature
y = df['churn'] # target 

In [5]:
print(x.dtypes)

tenure                               int64
age                                  int64
address                              int64
income                               int64
employ                               int64
reside                               int64
lninc                              float64
total_monthly_bill                 float64
total_bill                         float64
total_log_bill                     float64
region_Zone 1                        int64
region_Zone 2                        int64
region_Zone 3                        int64
marital_Married                      int64
marital_Unmarried                    int64
ed_College degree                    int64
ed_Did not complete high school      int64
ed_High school degree                int64
ed_Post-undergraduate degree         int64
ed_Some college                      int64
retire_No                            int64
retire_Yes                           int64
gender_Female                        int64
gender_Male

In [6]:
## We are using standard scaler as Variables that are measured at different scales do not contribute equally to 
#the model fitting & model learned function and might end up creating a bias.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(x)
X_train, X_test, Y_train, y_test = train_test_split(X_scaled, y,stratify=y, test_size=0.2,random_state=1)


In [7]:
## Logistic Regression 
log_reg = LogisticRegression()

log_reg.fit(X_train,Y_train)

LogisticRegression()

In [8]:
## Creating function that will evaluate model. 
## It will help us to reduced the code size and increase reusability of code

def evaluate_model(model,x_train,y_train,x_test,y_test,fit=False):
    '''
    Model Evaluation for Classifier
    :param  model : model object 
    :param x_train: Train features
    :param y_train: Train Target 
    :param x_test: Test features
    :param y_test: Test Target 
    :param fit bool : True if model is already fited else false

    :return: Train and Test Classification report and AUC- ROC Graph
    '''
    if fit == False:
        model.fit(x_train,y_train)
    train_pred=model.predict(x_train)
    print("Training report")
    print(classification_report(y_train, train_pred))    
    
    print("Testing report")
    test_pred=model.predict(x_test)    
    print(classification_report(y_test, test_pred))

# evaluating the model 
evaluate_model(log_reg,X_train,Y_train,X_test,y_test,fit=True)

model_score_log_reg= log_reg.score(X_test, y_test)
print('Test Score: ',model_score_log_reg)

Training report
              precision    recall  f1-score   support

           0       0.82      0.91      0.86       581
           1       0.66      0.45      0.54       219

    accuracy                           0.79       800
   macro avg       0.74      0.68      0.70       800
weighted avg       0.77      0.79      0.77       800

Testing report
              precision    recall  f1-score   support

           0       0.82      0.91      0.87       145
           1       0.68      0.49      0.57        55

    accuracy                           0.80       200
   macro avg       0.75      0.70      0.72       200
weighted avg       0.78      0.80      0.78       200

Test Score:  0.795



- we need to check recall, precision & f1 score for the minority class, and it's quite evident that the precision, recall & f1 score is too low for Class 1, i.e. churned customers.


In [9]:
##Base model on SMOTE :
## Using SMOTE To balance Training dataset 
# transform the dataset

oversample = SMOTE()
x_train, y_train = oversample.fit_resample(X_train, Y_train)

log_reg_smote = LogisticRegression()

log_reg_smote.fit(x_train,y_train)

print("Training data size")
print(x_train.shape)
print(y_train.value_counts())
evaluate_model(log_reg_smote,x_train,y_train,X_test,y_test,fit=True)

model_score_log_reg_smote= log_reg_smote.score(X_test, y_test)
print('Test Score: ',model_score_log_reg_smote)

Training data size
(1162, 54)
0    581
1    581
Name: churn, dtype: int64
Training report
              precision    recall  f1-score   support

           0       0.77      0.73      0.75       581
           1       0.75      0.78      0.77       581

    accuracy                           0.76      1162
   macro avg       0.76      0.76      0.76      1162
weighted avg       0.76      0.76      0.76      1162

Testing report
              precision    recall  f1-score   support

           0       0.90      0.71      0.80       145
           1       0.51      0.80      0.62        55

    accuracy                           0.73       200
   macro avg       0.71      0.76      0.71       200
weighted avg       0.80      0.73      0.75       200

Test Score:  0.735


In [10]:
# %
#applying smoteenn to balance our data set 
sm = SMOTEENN()
X_resampled, y_resampled = sm.fit_resample(x,y)

xr_train,xr_test,yr_train,yr_test=train_test_split(X_resampled, y_resampled,test_size=0.2)

In [11]:
#logistic regression with smoteenn
model_LR_smoteen=LogisticRegression()

model_LR_smoteen.fit(xr_train,yr_train)
model1_score_r = model_LR_smoteen.score(xr_test, yr_test)

evaluate_model(model_LR_smoteen,xr_train,yr_train,xr_test,yr_test,fit=True)
print('Test Score :',model1_score_r)


Training report
              precision    recall  f1-score   support

           0       0.88      0.88      0.88       229
           1       0.91      0.91      0.91       314

    accuracy                           0.90       543
   macro avg       0.90      0.90      0.90       543
weighted avg       0.90      0.90      0.90       543

Testing report
              precision    recall  f1-score   support

           0       0.80      0.85      0.82        52
           1       0.90      0.87      0.88        84

    accuracy                           0.86       136
   macro avg       0.85      0.86      0.85       136
weighted avg       0.86      0.86      0.86       136

Test Score : 0.8602941176470589


In [12]:
#Support Vector Machine 
svm=SVC()
svm.fit(X_train,Y_train)

SVC()

In [13]:
# %
evaluate_model(svm,X_train,Y_train,X_test,y_test,fit=True)

model_score_svm= svm.score(X_test, y_test)
print('Test Score: ',model_score_svm)

Training report
              precision    recall  f1-score   support

           0       0.84      0.96      0.90       581
           1       0.85      0.53      0.65       219

    accuracy                           0.84       800
   macro avg       0.84      0.74      0.77       800
weighted avg       0.84      0.84      0.83       800

Testing report
              precision    recall  f1-score   support

           0       0.79      0.94      0.86       145
           1       0.68      0.35      0.46        55

    accuracy                           0.78       200
   macro avg       0.73      0.64      0.66       200
weighted avg       0.76      0.78      0.75       200

Test Score:  0.775


In [14]:
# %
#applying SMOTE
svm_smote=SVC()

#fitting the scv smote model
svm_smote.fit(x_train,y_train)

#evaluating the model
evaluate_model(svm_smote,X_train,Y_train,X_test,y_test,fit=True)

model_score_svm_smote= svm_smote.score(X_test, y_test)
print('Test Score: ',model_score_svm_smote)

Training report
              precision    recall  f1-score   support

           0       0.94      0.87      0.91       581
           1       0.72      0.86      0.78       219

    accuracy                           0.87       800
   macro avg       0.83      0.87      0.85       800
weighted avg       0.88      0.87      0.87       800

Testing report
              precision    recall  f1-score   support

           0       0.82      0.77      0.79       145
           1       0.48      0.56      0.52        55

    accuracy                           0.71       200
   macro avg       0.65      0.66      0.65       200
weighted avg       0.73      0.71      0.72       200

Test Score:  0.71


In [15]:
#Support Vector Machine with smoteenn

model_scv_smoteen=SVC()

model_scv_smoteen.fit(xr_train,yr_train)
model2_score_r = model_scv_smoteen.score(xr_test, yr_test)

evaluate_model(model_scv_smoteen,xr_train,yr_train,xr_test,yr_test,fit=True)
print('Test Score :',model2_score_r)


Training report
              precision    recall  f1-score   support

           0       0.78      0.49      0.60       229
           1       0.71      0.90      0.79       314

    accuracy                           0.73       543
   macro avg       0.74      0.70      0.70       543
weighted avg       0.74      0.73      0.71       543

Testing report
              precision    recall  f1-score   support

           0       0.71      0.46      0.56        52
           1       0.73      0.88      0.80        84

    accuracy                           0.72       136
   macro avg       0.72      0.67      0.68       136
weighted avg       0.72      0.72      0.70       136

Test Score : 0.7205882352941176


- SMOTE ENN Performed better for linear models 


In [16]:
# %%
# Random Forest
rf_clf = RandomForestClassifier(n_jobs=-1)
rf_clf.fit(x_train,y_train)
evaluate_model(rf_clf,x_train,y_train,X_test,y_test,fit=True)

model_score_rand_clf = rf_clf.score(X_test, y_test)
print('Test Score: ',model_score_rand_clf)

Training report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       581
           1       1.00      1.00      1.00       581

    accuracy                           1.00      1162
   macro avg       1.00      1.00      1.00      1162
weighted avg       1.00      1.00      1.00      1162

Testing report
              precision    recall  f1-score   support

           0       0.84      0.86      0.85       145
           1       0.60      0.58      0.59        55

    accuracy                           0.78       200
   macro avg       0.72      0.72      0.72       200
weighted avg       0.78      0.78      0.78       200

Test Score:  0.78


In [17]:
## Hyperparameter tunining 
# we are tuning three hyperparameters right now, we are passing the different values for both parameters
grid_param = {
    "n_estimators" : [90,100,115,130],
    'criterion': ['gini', 'entropy'],
    'max_depth' : range(2,20,1),
    'min_samples_leaf' : range(1,10,1),
    'min_samples_split': range(2,10,1),
    'max_features' : ['auto','log2']
}
random_search = RandomizedSearchCV(estimator=rf_clf,param_distributions=grid_param,cv=5,n_jobs =-1,verbose = 3)
random_search.fit(x_train,y_train)
# %%
# %%
print(random_search.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
{'n_estimators': 115, 'min_samples_split': 6, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 19, 'criterion': 'gini'}


In [18]:
rand_clf_tune = RandomForestClassifier(criterion= 'entropy',
 max_depth = 14,
 max_features = 'log2',
 min_samples_leaf = 1,
 min_samples_split= 4,
 n_estimators = 115,random_state=6)

rand_clf_tune.fit(x_train,y_train)
evaluate_model(rand_clf_tune,x_train,y_train,X_test,y_test,fit=True)

model_score_rand_clf_tune = rand_clf_tune.score(X_test, y_test)
print('Test Score: ',model_score_rand_clf_tune)


Training report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       581
           1       1.00      1.00      1.00       581

    accuracy                           1.00      1162
   macro avg       1.00      1.00      1.00      1162
weighted avg       1.00      1.00      1.00      1162

Testing report
              precision    recall  f1-score   support

           0       0.85      0.83      0.84       145
           1       0.58      0.60      0.59        55

    accuracy                           0.77       200
   macro avg       0.71      0.72      0.71       200
weighted avg       0.77      0.77      0.77       200

Test Score:  0.77


In [19]:
#%
# Random Forest with smoteenn

model_rf_smoteen=RandomForestClassifier(criterion= 'entropy',
 max_depth = 14,
 max_features = 'log2',
 min_samples_leaf = 1,
 min_samples_split= 4,
 n_estimators = 115,random_state=6)

In [20]:
model_rf_smoteen.fit(xr_train,yr_train)
yr_predict4 = model_rf_smoteen.predict(xr_test)
model3_score_r = model_rf_smoteen.score(xr_test, yr_test)

evaluate_model(model_scv_smoteen,xr_train,yr_train,xr_test,yr_test,fit=True)
print('Test Score :',model3_score_r)


Training report
              precision    recall  f1-score   support

           0       0.78      0.49      0.60       229
           1       0.71      0.90      0.79       314

    accuracy                           0.73       543
   macro avg       0.74      0.70      0.70       543
weighted avg       0.74      0.73      0.71       543

Testing report
              precision    recall  f1-score   support

           0       0.71      0.46      0.56        52
           1       0.73      0.88      0.80        84

    accuracy                           0.72       136
   macro avg       0.72      0.67      0.68       136
weighted avg       0.72      0.72      0.70       136

Test Score : 0.9264705882352942


In [21]:
# %
# Gossian Naive Bayes 
nb_clf=GaussianNB()
nb_clf.fit(x_train,y_train)
evaluate_model(nb_clf,x_train,y_train,X_test,y_test,fit=True)

model_score_nb_clf = nb_clf.score(X_test, y_test)
print(model_score_nb_clf)


Training report
              precision    recall  f1-score   support

           0       0.77      0.61      0.68       581
           1       0.68      0.82      0.74       581

    accuracy                           0.72      1162
   macro avg       0.72      0.72      0.71      1162
weighted avg       0.72      0.72      0.71      1162

Testing report
              precision    recall  f1-score   support

           0       0.93      0.59      0.73       145
           1       0.45      0.89      0.60        55

    accuracy                           0.68       200
   macro avg       0.69      0.74      0.66       200
weighted avg       0.80      0.68      0.69       200

0.675


In [22]:
# Gossian Naive Bayes with smoteenn

model_nb_smoteen=GaussianNB()

model_nb_smoteen.fit(xr_train,yr_train)
yr_predict3 = model_nb_smoteen.predict(xr_test)
model4_score_r = model_nb_smoteen.score(xr_test, yr_test)

evaluate_model(model_scv_smoteen,xr_train,yr_train,xr_test,yr_test,fit=True)
print('Test Score :',model4_score_r)


Training report
              precision    recall  f1-score   support

           0       0.78      0.49      0.60       229
           1       0.71      0.90      0.79       314

    accuracy                           0.73       543
   macro avg       0.74      0.70      0.70       543
weighted avg       0.74      0.73      0.71       543

Testing report
              precision    recall  f1-score   support

           0       0.71      0.46      0.56        52
           1       0.73      0.88      0.80        84

    accuracy                           0.72       136
   macro avg       0.72      0.67      0.68       136
weighted avg       0.72      0.72      0.70       136

Test Score : 0.7794117647058824


### Pickling the model


In [23]:
import pickle
pickle_out=open("model_rf_smoteen.pkl","wb")
pickle.dump(model_rf_smoteen,pickle_out)
pickle_out.close()